In [7]:
import torch
import onlinehd
import torchvision.datasets as datasets
import numpy as np
import sklearn
from sklearn import *
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
# open/read the .csv dataset with panda
coll = pd.read_csv("./data/original_college_data.csv") 

In [9]:
# make categorical variables numeric
coll["type_school"]= coll["type_school"].replace("Academic", 0)
coll["type_school"]= coll["type_school"].replace("Vocational", 1)
coll["school_accreditation"]= coll["school_accreditation"].replace("A", 0)
coll["school_accreditation"]= coll["school_accreditation"].replace("B", 1)
coll["gender"]= coll["gender"].replace("Male", 1)
coll["gender"]= coll["gender"].replace("Female", 0)
coll["interest"]= coll["interest"].replace("Not Interested", -2)
coll["interest"]= coll["interest"].replace("Less Interested", -1)
coll["interest"]= coll["interest"].replace("Uncertain", 0)
coll["interest"]= coll["interest"].replace("Quiet Interested", 1)
coll["interest"]= coll["interest"].replace("Very Interested", 2)
coll["residence"]= coll["residence"].replace("Urban", 1)
coll["residence"]= coll["residence"].replace("Rural", 0)
coll["parent_was_in_college"]= coll["parent_was_in_college"].replace(True, 1)
coll["parent_was_in_college"]= coll["parent_was_in_college"].replace(False, 0)
coll["in_college"]= coll["in_college"].replace("TRUE", 1)
coll["in_college"]= coll["in_college"].replace("FALSE", 0)

In [10]:
# make it a numpy array so numpy can read it
coll = np.array(coll)

# first 8 columns of matrix are considered parameters
# the last one is considered to be the class
X = np.array(coll[:, :10])
Y = np.array(coll[:, 10])

In [11]:
X, X_test, Y, Y_test = sklearn.model_selection.train_test_split(X, Y)
X = torch.Tensor(X.astype('float64'))
Y = torch.Tensor(Y.astype('int')).long()
X_test = torch.Tensor(X_test.astype('float64'))
Y_test = torch.Tensor(Y_test.astype('int64')).long()

print("X train shape", X.shape, "Y train shape", Y.shape)
print("X test shape", X_test.shape, "Y Test shape", Y_test.shape)

X train shape torch.Size([750, 10]) Y train shape torch.Size([750])
X test shape torch.Size([250, 10]) Y Test shape torch.Size([250])


In [12]:
from fhrr import FHRR_Model

In [13]:
classes = len(np.unique(Y))
features = X.shape[1]
print("Number of classes", classes, '\nNumber of features', features)

Number of classes 2 
Number of features 10


In [14]:
dim = 10000
model = onlinehd.OnlineHD(classes, features, dim)
model.fit(X, Y, epochs=100) 

/opt/homebrew/Caskroom/miniforge/base/envs/hd/lib/python3.10/site-packages/onlinehd/encoder.py:61: UserWarning: An output with one or more elements was resized since it had shape [8, 10000], which does not match the required output shape [6, 10000]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Resize.cpp:24.)
  torch.matmul(x[i:i+bsize], self.basis.T, out=temp)


In [15]:
X_test = model.encode(X_test)
preds = model.predict(X_test, encoded=True)
acc = sklearn.metrics.accuracy_score(Y_test.numpy(), preds)
print(acc)

0.676


/opt/homebrew/Caskroom/miniforge/base/envs/hd/lib/python3.10/site-packages/onlinehd/encoder.py:61: UserWarning: An output with one or more elements was resized since it had shape [3, 10000], which does not match the required output shape [1, 10000]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Resize.cpp:24.)
  torch.matmul(x[i:i+bsize], self.basis.T, out=temp)
